In [7]:
%pip install langchain
%pip install python-dotenv
%pip install -qU langchain-openai
%pip install langchain-chroma

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [31]:
# export LANGCHAIN_TRACING_V2="true"
# export LANGCHAIN_API_KEY="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"

# import getpass
# import os

# os.environ["LANGCHAIN_TRACING_V2"] = "true"
# os.environ["LANGCHAIN_API_KEY"] = getpass.getpass()

In [32]:
from langchain_core.documents import Document

documents = [
    Document(
        page_content="Dogs are great companions, known for their loyalty and friendliness.",
        metadata={"source": "mammal-pets-doc"},
    ),
    Document(
        page_content="Cats are independent pets that often enjoy their own space.",
        metadata={"source": "mammal-pets-doc"},
    ),
    Document(
        page_content="Goldfish are popular pets for beginners, requiring relatively simple care.",
        metadata={"source": "fish-pets-doc"},
    ),
    Document(
        page_content="Parrots are intelligent birds capable of mimicking human speech.",
        metadata={"source": "bird-pets-doc"},
    ),
    Document(
        page_content="Rabbits are social animals that need plenty of space to hop around.",
        metadata={"source": "mammal-pets-doc"},
    ),
]

In [33]:
import os
import getpass
from dotenv import load_dotenv
from langchain_core.messages import HumanMessage
from langchain_core.messages import AIMessage
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_chroma import Chroma
from langchain_openai import ChatOpenAI
from langchain_openai import OpenAIEmbeddings

# Load environment variables from .env file
load_dotenv()

open_ai_key = os.getenv('OPEN_AI_KEY')
open_ai_org = os.getenv('OPEN_AI_ORG')

os.environ["OPENAI_API_KEY"] = open_ai_key

model = ChatOpenAI(
    openai_api_key=open_ai_key,
    openai_organization=open_ai_org,
    max_tokens=1000,
    model="gpt-4"
)

embeddings = OpenAIEmbeddings(
    openai_api_key=open_ai_key,
    openai_organization=open_ai_org
)

vectorstore = Chroma.from_documents(
    documents,
    embedding=embeddings,
)

# vectorstore = Chroma.from_documents(
#     documents,
#     embedding=OpenAIEmbeddings(),
# )


In [34]:
vectorstore.similarity_search("cat")

[Document(page_content='Cats are independent pets that often enjoy their own space.', metadata={'source': 'mammal-pets-doc'}),
 Document(page_content='Cats are independent pets that often enjoy their own space.', metadata={'source': 'mammal-pets-doc'}),
 Document(page_content='Cats are independent pets that often enjoy their own space.', metadata={'source': 'mammal-pets-doc'}),
 Document(page_content='Dogs are great companions, known for their loyalty and friendliness.', metadata={'source': 'mammal-pets-doc'})]

In [22]:
await vectorstore.asimilarity_search("cat")

[Document(page_content='Cats are independent pets that often enjoy their own space.', metadata={'source': 'mammal-pets-doc'}),
 Document(page_content='Cats are independent pets that often enjoy their own space.', metadata={'source': 'mammal-pets-doc'}),
 Document(page_content='Dogs are great companions, known for their loyalty and friendliness.', metadata={'source': 'mammal-pets-doc'}),
 Document(page_content='Dogs are great companions, known for their loyalty and friendliness.', metadata={'source': 'mammal-pets-doc'})]

In [16]:
vectorstore.similarity_search_with_score("Goldfish")

[(Document(page_content='Goldfish are popular pets for beginners, requiring relatively simple care.', metadata={'source': 'fish-pets-doc'}),
  0.192320317029953),
 (Document(page_content='Goldfish are popular pets for beginners, requiring relatively simple care.', metadata={'source': 'fish-pets-doc'}),
  0.19268865883350372),
 (Document(page_content='Dogs are great companions, known for their loyalty and friendliness.', metadata={'source': 'mammal-pets-doc'}),
  0.457475483417511),
 (Document(page_content='Dogs are great companions, known for their loyalty and friendliness.', metadata={'source': 'mammal-pets-doc'}),
  0.4581882953643799)]

In [35]:
# embedding = OpenAIEmbeddings().embed_query("cat")

embedding_cat = OpenAIEmbeddings(
    openai_api_key=open_ai_key,
    openai_organization=open_ai_org
).embed_query("dog")

vectorstore.similarity_search_by_vector(embedding_cat)

[Document(page_content='Dogs are great companions, known for their loyalty and friendliness.', metadata={'source': 'mammal-pets-doc'}),
 Document(page_content='Dogs are great companions, known for their loyalty and friendliness.', metadata={'source': 'mammal-pets-doc'}),
 Document(page_content='Dogs are great companions, known for their loyalty and friendliness.', metadata={'source': 'mammal-pets-doc'}),
 Document(page_content='Cats are independent pets that often enjoy their own space.', metadata={'source': 'mammal-pets-doc'})]

# Retriever

In [36]:
from typing import List

from langchain_core.documents import Document
from langchain_core.runnables import RunnableLambda

retriever = RunnableLambda(vectorstore.similarity_search).bind(k=1)  # select top result

retriever.batch(["cat", "shark"])

--- Logging error ---
Traceback (most recent call last):
  File "/Users/apirak/Library/Caches/pypoetry/virtualenvs/deal-droid-Yh1ExAlx-py3.12/lib/python3.12/site-packages/langchain_core/callbacks/manager.py", line 2009, in _configure
    handler = LangChainTracer(
              ^^^^^^^^^^^^^^^^
  File "/Users/apirak/Library/Caches/pypoetry/virtualenvs/deal-droid-Yh1ExAlx-py3.12/lib/python3.12/site-packages/langchain_core/tracers/langchain.py", line 91, in __init__
    self.client = client or get_client()
                            ^^^^^^^^^^^^
  File "/Users/apirak/Library/Caches/pypoetry/virtualenvs/deal-droid-Yh1ExAlx-py3.12/lib/python3.12/site-packages/langchain_core/tracers/langchain.py", line 54, in get_client
    _CLIENT = Client()
              ^^^^^^^^
  File "/Users/apirak/Library/Caches/pypoetry/virtualenvs/deal-droid-Yh1ExAlx-py3.12/lib/python3.12/site-packages/langsmith/client.py", line 540, in __init__
    _validate_api_key_if_hosted(self.api_url, self.api_key)
  File "/U

[[Document(page_content='Cats are independent pets that often enjoy their own space.', metadata={'source': 'mammal-pets-doc'})],
 [Document(page_content='Goldfish are popular pets for beginners, requiring relatively simple care.', metadata={'source': 'fish-pets-doc'})]]

In [20]:
retriever = vectorstore.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 1},
)

retriever.batch(["cat", "shark"])

[[Document(page_content='Cats are independent pets that often enjoy their own space.', metadata={'source': 'mammal-pets-doc'})],
 [Document(page_content='Goldfish are popular pets for beginners, requiring relatively simple care.', metadata={'source': 'fish-pets-doc'})]]

# Retriever for OpenAI

In [24]:
%pip install -qU langchain-openai

Note: you may need to restart the kernel to use updated packages.


In [26]:
import getpass
import os

from langchain_openai import ChatOpenAI

llm = ChatOpenAI(
    openai_api_key=open_ai_key,
    openai_organization=open_ai_org,
    max_tokens=1000,
    model="gpt-4"
)

In [37]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough

message = """
Answer this question using the provided context only.

{question}

Context:
{context}
"""

prompt = ChatPromptTemplate.from_messages([("human", message)])

rag_chain = {"context": retriever, "question": RunnablePassthrough()} | prompt | llm

[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], template='\nAnswer this question using the provided context only.\n\n{question}\n\nContext:\n{context}\n'))]


In [28]:
response = rag_chain.invoke("tell me about cats")

print(response.content)

Cats are independent pets that often enjoy their own space.
